In [ ]:
from imitation.data.types import Transitions
import numpy as np
import pickle

In [ ]:
file2 = open("MARS_AIRL_expert_transitions", "rb")
demos = pickle.load(file2)
file2.close()

In [ ]:
type(demos)

In [ ]:
from imitation.algorithms.adversarial.airl import AIRL
from imitation.rewards.reward_nets import BasicShapedRewardNet
from imitation.util.networks import RunningNorm
from imitation.util.util import make_vec_env
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import DDPG
from stable_baselines3 import SAC, A2C
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

import gym
from envs.pendulumV21 import PendulumEnv as pendulum


env = pendulum()
venv = make_vec_env(pendulum, n_envs=8)

n_actions = env.action_space.shape[-1]
print(env.action_space)
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

learner = SAC("MlpPolicy", venv, action_noise=action_noise, verbose=1, device='cpu')
# learner = A2C("MlpPolicy", venv, verbose=1)#, device='cpu')
reward_net = BasicShapedRewardNet(
    venv.observation_space, venv.action_space, normalize_input_layer=RunningNorm
)


In [ ]:
airl_trainer = AIRL(
    demonstrations=demos,
    demo_batch_size=64,
    gen_replay_buffer_capacity=2048,
    n_disc_updates_per_round=4,
    venv=venv,
    gen_algo=learner,
    reward_net=reward_net,
    allow_variable_horizon=True
)

# learner_rewards_before_training, _ = evaluate_policy(
#      learner, venv, 1, return_episode_rewards=True
# )
airl_trainer.train(300000)  # Note: set to 300000 for better results
# learner_rewards_after_training, _ = evaluate_policy(
#     learner, venv, 100, return_episode_rewards=True
# )

In [ ]:
learner.save("./saved_models/sac_airl_300k")